# Import needed libraries

In [1]:
import os
#import matplotlib.pyplot as plt
import re
import numpy as np
from path import Path
import csv
from pandas import *
import pandas as pd
import sys
import ast

In [2]:
from func_search import *
from func_taux_transm import *
from func_def_class import *
from func_res import *
from func_parsing_logfile import *
from func_parsing_csvfile import *

# Parse data file into specific log files

In [3]:
#### PARSING DES LOGS DANS DES SOUS FICHIER TXT
sep_data_addr("data/exp_10.log")
sep_data_event("data/exp_10.log")

# Create paquet transmission and reception dataframes

In [7]:
#### CREATION DES STRUCT PANDAS
#fichier tx
df_tx = create_df_tx("data/parsed/event/data_STAT_PK_TX.log")
df_tx.to_csv('data_csv/pkt_tx.csv',index=False)

In [8]:
# fichier rx
df_rx = create_df_rx("data/parsed/event/data_STAT_PK_RX.log", "data/parsed/event/data_STAT_ACK_RX.log")
df_rx.to_csv('data_csv/pkt_rx.csv',index=False)

# Complete the transmission dataframe

In [9]:
#Change some columns type for the nexts operations
df_tx = pd.read_csv(r'data_csv/pkt_tx.csv', dtype=str)
df_tx["asn"] = df_tx["asn"].astype(int)
df_tx["numTxAttempts"] = df_tx["numTxAttempts"].astype(int)
df_tx["trackinstance"] = df_tx["trackinstance"].astype(int)
df_tx = df_tx.fillna('')
df_rx = pd.read_csv(r'data_csv/pkt_rx.csv', dtype=str)
df_rx["asn"] = df_rx["asn"].astype(int)

In [11]:
#fill the reception columns in the tx df to know what paquet was received and ACK
df_tx = fill_succes_tx(df_tx,df_rx)
df_tx.to_csv('data_csv/pkt_tx.csv',index=False)

# Create the reservation dataframe

In [12]:
#creer la df qui contient les infos sur les res
df_res_step=create_df_step_res("data/parsed/event/data_OTHER_ParserPrintf.log")
df_res_step.to_csv('data_csv/res_step.csv',index=False)

In [14]:
# fichiers res
df_res=create_df_res("data/parsed/event/data_OTHER_ParserPrintf.log")
df_res.to_csv('data_csv/res.csv',index=False)

# create parent/son dataframe

In [15]:
#créer les fichiers parent et fils 
df_parent = create_df_parent("data/parsed/event/data_OTHER_ParserPrintf.log")
df_parent.to_csv('data_csv/parent.csv',index=False)

In [16]:
df_parent = pd.read_csv(r'data_csv/parent.csv', dtype=str)
df_parent["asn"] = df_parent["asn"].astype(int)
df_parent = df_parent.fillna('')

In [17]:
df_fils = create_df_sons(df_parent)
df_fils.to_csv('data_csv/sons.csv',index=False)

In [18]:
df_fils = pd.read_csv(r'data_csv/sons.csv', dtype=str)
df_fils["asn"] = df_fils["asn"].astype(int)

In [19]:
df_res = pd.read_csv(r'data_csv/res.csv', dtype=str)
df_res["numAttempts"] = df_res["numAttempts"].astype(int)
#df_res["nb_sibl"] = df_res["nb_sibl"].astype(int)
df_res["succes"] = df_res["succes"].astype(int)
df_res["state"] = df_res["state"].astype(int)
df_res["asn creation"] = df_res["asn creation"].astype(int)
df_res = df_res.fillna('')
df_res.sort_values(['asn creation', 'asn_req'], ascending=[True, True], inplace=True)

In [20]:
df_res = fill_nb_siblings(df_res, df_fils ,df_parent)
df_res.to_csv('data_csv/res.csv',index=False)

In [21]:
# a lancer après les siblings
df_res_sim = fill_simult(df_res, df_tx)
df_res_sim.to_csv('data_csv/res_sim.csv',index=False)

In [22]:
df_res = pd.read_csv(r'data_csv/res_sim.csv', dtype=str)
df_res["numAttempts"] = df_res["numAttempts"].astype(int)
df_res["nb_sibl"] = df_res["nb_sibl"].astype(int)
df_res["succes"] = df_res["succes"].astype(int)
df_res["state"] = df_res["state"].astype(int)
df_res["asn creation"] = df_res["asn creation"].astype(int)
df_res = df_res.fillna('')
df_res.sort_values(['asn creation', 'asn_req', 'asn_tx'], ascending=[True, True, True], inplace=True)